In [51]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [52]:
articles = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")
customers = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv")
transactions = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")

In [53]:
articles_for_merge = articles[['article_id', 'prod_name', 'product_type_name', 'product_group_name', 'index_name']]
articles_for_merge = transactions[['customer_id', 'article_id', 'price', 't_dat']].merge(articles_for_merge, on='article_id', how='left')
transaction_optn_df = articles_for_merge[articles_for_merge.customer_id.isin(articles_for_merge.customer_id.drop_duplicates().to_list()[:100])]
transaction_optn_df['customer_index'] = transaction_optn_df.customer_id.astype('category').cat.codes
transaction_optn_df['prod_index'] = transaction_optn_df.prod_name.astype('category').cat.codes

In [ ]:
# transaction_optn_df = transaction_optn_df.sort_values(by=['customer_index', 't_dat'], ascending=[True, True])
# transaction_optn_df['purchase_seq_ids'] = transaction_optn_df.groupby(['customer_id'])['article_id'].transform(lambda x: pd.factorize(x)[0]).tolist()
# t = transaction_optn_df[['customer_index','article_id','purchase_seq_ids']].drop_duplicates().pivot(index='customer_index', columns='purchase_seq_ids', values='article_id')

# t['journey']= t.values.tolist()
# actual_customerjourneys = (t.journey.apply(lambda y: [a for a in y if pd.notnull(a)])).to_frame().journey.apply(tuple).tolist()
# actual_customers = t.index.tolist()

In [54]:
transaction_optn_df.head(10)

# Segmenting the articles (items)

In [55]:
from kmodes.kmodes import KModes
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_articles = articles.select_dtypes(include=numerics)
numeric_articles_cols = numeric_articles.columns.tolist()
data = numeric_articles
data.index = data.article_id
data = data[data.columns[2:]].dropna()
# Building the model with 5 clusters
kmode = KModes(n_clusters=5, init = "random", n_init = 5, verbose=1)
clusters = kmode.fit_predict(data)
data['cluster_id'] = clusters
article_clusters = pd.DataFrame()
article_clusters['article_id'] = data.index
article_clusters['cluster_id'] = data.cluster_id.tolist()
article_clusters = article_clusters.drop_duplicates()
transaction_optn_df = transaction_optn_df.merge(article_clusters, on= ['article_id'], how= 'inner')
transaction_optn_df.cluster_id.value_counts()

# User Item Matrix

In [56]:
df = transaction_optn_df.groupby(['article_id', 'prod_name', 'product_type_name', 'product_group_name', 'index_name'])["customer_index"].count().reset_index(name="count")
df = df.sort_values(by=['count'], ascending = [False])
df_with_count = transaction_optn_df.merge(df, on= ['article_id', 'prod_name', 'product_type_name', 'product_group_name',
       'index_name'], how='inner')
customer_item_matrix = df_with_count.pivot_table(
    index='customer_index', 
    columns='article_id', 
    values='count',
    aggfunc='sum'
)
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)

In [57]:
from sklearn.metrics.pairwise import cosine_similarity
user_user_sim_matrix = pd.DataFrame(
    cosine_similarity(customer_item_matrix)
)
user_user_sim_matrix.columns = customer_item_matrix.index
user_user_sim_matrix['customer_index'] = customer_item_matrix.index
user_user_sim_matrix = user_user_sim_matrix.set_index('customer_index')

In [58]:
user_user_sim_matrix.loc[90].sort_values(ascending=False)
items_bought_by_A = set(customer_item_matrix.loc[90].iloc[
    customer_item_matrix.loc[90].to_numpy().nonzero()
].index)
items_bought_by_B = set(customer_item_matrix.loc[91].iloc[
    customer_item_matrix.loc[91].to_numpy().nonzero()
].index)
items_to_recommend_to_B = items_bought_by_A - items_bought_by_B

In [59]:
df.loc[
    df['article_id'].isin(items_to_recommend_to_B), 
    ['article_id', 'prod_name']
].drop_duplicates().set_index('article_id')